# API KEY

In [ ]:
# API Key Deepseek Chimera
# --> https://openrouter.ai/settings/keys
OPENROUTER_API_KEY = "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"  # <--  API KEY

In [36]:
PEXELS_API_KEY = "mAeOeBxoIDpJba5LKFHLBNfp4ektHkYJYLAQhemwTJEBX9rmjTsTJmo8"

# Library

In [18]:
import json
import os
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from gtts import gTTS

In [19]:
# Load FAISS Index
index = faiss.read_index("output/faiss_index.bin")

# Load Mapping sumber chunk
with open("output/chunk_mapping.json", "r", encoding="utf-8") as f:
    chunk_sources = json.load(f)

# Load ulang semua chunk text
with open("output/cleaned_text/epub_chunks_translated.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Gabungkan semua chunk
all_chunks = []
for book_title, chunks in data.items():
    for chunk in chunks:
        if chunk.strip():
            all_chunks.append(chunk)

# Load model embedding
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [20]:
def search_best_chunk(question, top_k=1):
    """Cari chunk paling relevan untuk pertanyaan."""
    question_embedding = model.encode([question]).astype('float32')
    D, I = index.search(question_embedding, top_k)

    results = []
    for idx in I[0]:
        if idx < len(all_chunks):
            results.append((chunk_sources[idx], all_chunks[idx]))
    return results


In [21]:
import requests

def summarize_with_deepseek(text, api_key=OPENROUTER_API_KEY):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    prompt = (
    "Summarize the following text into a compelling, clear, and natural narration "
    "that sounds like it's spoken in a 1-minute educational YouTube short. "
    "Use warm and engaging tone. Avoid robotic or overly formal phrasing. "
    "Write as if you're explaining to a general audience with no prior knowledge.\n\n"
    f"{text}"
    )

    data = {
        "model": "tngtech/deepseek-r1t-chimera:free",  # Updated model ID
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }

    response = requests.post(url, headers=headers, json=data)

    # Check if there is an error in the response
    if response.status_code != 200:
        print(f"❌ ERROR: {response.status_code} - {response.text}")
        return "[SUMMARY FAILED]"

    response_data = response.json()

    if 'choices' not in response_data:
        print(f"❌ ERROR: {response_data}")
        return "[SUMMARY FAILED]"

    summary = response_data['choices'][0]['message']['content']
    return summary


In [22]:
def text_to_speech(text, output_audio_path="output/answer_audio.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(output_audio_path)
    return output_audio_path


In [44]:
# Pertanyaan user
question = "What does Islam say about the importance of youth?"

# Step 1: Cari Chunk
results = search_best_chunk(question, top_k=1)
source, answer = results[0]

print(f"📖 Source: {source}")
print(f"📝 Original Answer: {answer}")

# Step 2: Summarize dengan DeepSeek
summarized_answer = summarize_with_deepseek(answer)
print(f"📝 Summarized Answer: {summarized_answer}")

# Step 3: Buat Audio
audio_path = text_to_speech(summarized_answer)



📖 Source: الأربعون الشبابية - Chunk 1
📝 Original Answer: Cover forty youth Book: Forty youth, author: Muhammad Khair Ramadan Yusuf, unknown, forty youth, Muhammad Khair Ramadan Muharram 1438 AH. Source: The Golden Comprehensive is anonymous in the name of God, the Most Gracious, the Most Merciful. And educate them to strengthen, and direct them to advocacy and jihad, such as Ali, Musab, Moaz, Ibn Masoud, Ibn Omar, and others, may God be pleased with them all. And their news is many in the Sunnah and the generous prophetic biography, and if the word (youth) and (the boys) did not mention and what they behave from them, they were the audience of the new religion and its fuel, and the movement of society and its activity and vitality, and the Messenger of God, peace and blessings of God be upon him, assigned them to great deeds, to feel the responsibility of the religion About him with senior leaders. I loved the collection of a group of hadiths in which the young and young people are men

In [ ]:
# import os
# from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip, ColorClip, concatenate_videoclips
# from moviepy.config import change_settings
# import textwrap

# change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})


# def split_text_to_chunks(text, max_chars=120):
#     """
#     Memecah teks menjadi potongan-potongan pendek agar cocok untuk satu layar.
#     """
#     return textwrap.wrap(text, width=max_chars)


# def create_video_with_audio_and_text(audio_path, text, output_path="output/final_video.mp4"):
#     # Load audio
#     audio = AudioFileClip(audio_path)
#     duration = audio.duration

#     # Pecah teks jadi beberapa bagian
#     chunks = split_text_to_chunks(text)
#     num_chunks = len(chunks)
#     chunk_duration = duration / num_chunks  # Durasi tiap segmen

#     video_clips = []

#     for i, chunk in enumerate(chunks):
#         background = ColorClip(size=(1280, 720), color=(0, 0, 0), duration=chunk_duration)

#         text_clip = TextClip(
#             txt=chunk,
#             fontsize=48,
#             color='white',
#             font='Arial-Bold',
#             size=(1100, None),
#             method='caption'
#         ).set_duration(chunk_duration).set_position(("center", "bottom"))

#         video_clip = CompositeVideoClip([background, text_clip])
#         video_clips.append(video_clip)

#     # Gabungkan semua klip
#     final_video = concatenate_videoclips(video_clips).set_audio(audio)

#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     final_video.write_videofile(output_path, fps=24)

# # Contoh pemanggilan (pastikan audio_path dan summarized_answer didefinisikan)
# create_video_with_audio_and_text(audio_path, summarized_answer)
# print("✅ Video berhasil dibuat.")

In [42]:
import os
import requests
from io import BytesIO
from PIL import Image
from moviepy.editor import AudioFileClip, TextClip, CompositeVideoClip, ImageClip, concatenate_videoclips
from moviepy.config import change_settings
import textwrap
import hashlib

change_settings({"IMAGEMAGICK_BINARY": r"C:\\Program Files\\ImageMagick-7.1.1-Q16-HDRI\\magick.exe"})

PEXELS_API_KEY = "mAeOeBxoIDpJba5LKFHLBNfp4ektHkYJYLAQhemwTJEBX9rmjTsTJmo8"

def split_text_to_chunks(text, max_chars=120):
    return textwrap.wrap(text, width=max_chars)

def sanitize_filename(text):
    return hashlib.md5(text.encode('utf-8')).hexdigest()

def download_single_image_from_pexels(query, index=0):
    headers = {
        "Authorization": PEXELS_API_KEY
    }
    response = requests.get(f"https://api.pexels.com/v1/search?query={query}&per_page=1", headers=headers)
    data = response.json()
    
    if "photos" in data and data["photos"]:
        image_url = data["photos"][0]["src"]["landscape"]
        img_data = requests.get(image_url).content
        image = Image.open(BytesIO(img_data)).resize((1280, 720))
        image_path = f"temp_bg_{sanitize_filename(query)}_{index}.jpg"
        image.save(image_path)
        return image_path
    else:
        # fallback background
        black_img_path = f"temp_bg_black_{index}.jpg"
        Image.new("RGB", (1280, 720), (0, 0, 0)).save(black_img_path)
        return black_img_path

def create_video_with_audio_and_text(audio_path, text, output_path="output/final_video.mp4"):
    audio = AudioFileClip(audio_path)
    duration = audio.duration

    chunks = split_text_to_chunks(text)
    num_chunks = len(chunks)
    chunk_duration = duration / num_chunks

    video_clips = []
    temp_image_paths = []

    for i, chunk in enumerate(chunks):
        query = chunk.split(".")[0]
        bg_path = download_single_image_from_pexels(query, index=i)
        temp_image_paths.append(bg_path)  # simpan path untuk dihapus nanti

        background = ImageClip(bg_path).set_duration(chunk_duration)

        text_clip = TextClip(
            txt=chunk,
            fontsize=48,
            color='white',
            font='Arial-Bold',
            size=(1100, None),
            method='caption'
        ).set_duration(chunk_duration).set_position(("center", "bottom"))

        video_clip = CompositeVideoClip([background, text_clip])
        video_clips.append(video_clip)

    final_video = concatenate_videoclips(video_clips).set_audio(audio)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    final_video.write_videofile(output_path, fps=24)

    # Hapus semua gambar setelah selesai
    for path in temp_image_paths:
        if os.path.exists(path):
            os.remove(path)

# Contoh pemanggilan
create_video_with_audio_and_text(audio_path, summarized_answer)
print("✅ Video berhasil dibuat & gambar sementara telah dihapus.")


Moviepy - Building video output/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output/final_video.mp4



Moviepy - Done !
Moviepy - video ready output/final_video.mp4
✅ Video berhasil dibuat & gambar sementara telah dihapus.
